# The impact of dimensionality reduction in sentiment analysis classification

## Initialization

In [1]:
# Basic libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Time library
import time

# Dataset manipulation/vectorization libraries
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer

# Text processing libraries
import re
import nltk
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

# Dimensionality Reduction Algorithms
from sklearn.decomposition import TruncatedSVD

# Supervised classifiers.
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier

# Evaluation libraries
from sklearn.metrics import classification_report


[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\Leo\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\Leo\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     C:\Users\Leo\AppData\Roaming\nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


## Classifiers

In [2]:
models = [
        KNeighborsClassifier(10),
        LogisticRegression(max_iter=300),
        DecisionTreeClassifier(),
        RandomForestClassifier(n_estimators=100),
        SVC(kernel='rbf', C = 1),
        MLPClassifier(activation='relu', hidden_layer_sizes=(128, 16), random_state=42),
]
models

[KNeighborsClassifier(n_neighbors=10),
 LogisticRegression(max_iter=300),
 DecisionTreeClassifier(),
 RandomForestClassifier(),
 SVC(C=1),
 MLPClassifier(hidden_layer_sizes=(128, 16), random_state=42)]

## Dataset: IMDB


In [3]:
from sklearn import preprocessing

df = pd.read_csv("datasets\\tweets.csv", encoding = "utf-8")

df = df.rename(columns={"text": "text", "airline_sentiment": "source"}, errors='raise')

le = preprocessing.LabelEncoder()
df['source'] = le.fit_transform(df['source'])


df.head(10)


,tweet_id,source,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone
0,570306133677760513,1,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada)
1,570301130888122368,2,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada)
2,570301083672813571,1,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada)
3,570301031407624196,0,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada)
4,570300817074462722,0,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada)
5,570300767074181121,0,1.0000,Can't Tell,0.6842,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica seriously would pay $30 a fligh...,NaN,2015-02-24 11:14:33 -0800,NaN,Pacific Time (US & Canada)
6,570300616901320704,2,0.6745,NaN,0.0000,Virgin America,NaN,cjmcginnis,NaN,0,"@VirginAmerica yes, nearly every time I fly VX...",NaN,2015-02-24 11:13:57 -0800,San Francisco CA,Pacific Time (US & Canada)
7,570300248553349120,1,0.6340,NaN,NaN,Virgin America,NaN,pilot,NaN,0,@VirginAmerica Really missed a prime opportuni...,NaN,2015-02-24 11:12:29 -0800,Los Angeles,Pacific Time (US & Canada)
8,570299953286942721,2,0.6559,NaN,NaN,Virgin America,NaN,dhepburn,NaN,0,"@virginamerica Well, I didn't…but NOW I DO! :-D",NaN,2015-02-24 11:11:19 -0800,San Diego,Pacific Time (US & Canada)
9,570295459631263746,2,1.0000,NaN,NaN,Virgin America,NaN,YupitsTate,NaN,0,"@VirginAmerica it was amazing, and arrived an ...",NaN,2015-02-24 10:53:27 -0800,Los Angeles,Eastern Time (US & Canada)


## Text cleaning

In [4]:
def textcleaner_lemmas(text):
    ''' Takes in raw unformatted text and strips punctuation, removes whitespace,
    strips numbers, tokenizes and stems.
    Returns string of processed text to be used into CountVectorizer
    '''
    # Lowercase and strip everything except words
    cleaner = re.sub(r"[^a-zA-Z ]+", ' ', text.lower())
    # Tokenize
    cleaner = word_tokenize(cleaner)
    clean = []
    for w in cleaner:
        # filter out stopwords
        if w not in stopWords:
            # filter out short words
            if len(w)>2:
                # lemmatizer 
                clean.append(lemmatizer.lemmatize(w))
    return ' '.join(clean)


In [5]:
lemmatizer = WordNetLemmatizer()

stopWords = set(stopwords.words('english'))

t0 = time.time()
'''
label = []
for i in df['source']:
    if df['source'] == 1:
        label.append('positive')
    else:
        label.append('negative')
#    label.append(dataset.target_names[i])

df['label'] = label
'''
df['clean_text'] = df.text.apply(lambda x: textcleaner_lemmas(x))
t1 = time.time()

print("Text cleaning duration:", t1 - t0)
df.head()


Text cleaning duration: 3.3940000534057617


,tweet_id,source,airline_sentiment_confidence,negativereason,negativereason_confidence,airline,airline_sentiment_gold,name,negativereason_gold,retweet_count,text,tweet_coord,tweet_created,tweet_location,user_timezone,clean_text
0,570306133677760513,1,1.0000,NaN,NaN,Virgin America,NaN,cairdin,NaN,0,@VirginAmerica What @dhepburn said.,NaN,2015-02-24 11:35:52 -0800,NaN,Eastern Time (US & Canada),virginamerica dhepburn said
1,570301130888122368,2,0.3486,NaN,0.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica plus you've added commercials t...,NaN,2015-02-24 11:15:59 -0800,NaN,Pacific Time (US & Canada),virginamerica plus added commercial experience...
2,570301083672813571,1,0.6837,NaN,NaN,Virgin America,NaN,yvonnalynn,NaN,0,@VirginAmerica I didn't today... Must mean I n...,NaN,2015-02-24 11:15:48 -0800,Lets Play,Central Time (US & Canada),virginamerica today must mean need take anothe...
3,570301031407624196,0,1.0000,Bad Flight,0.7033,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica it's really aggressive to blast...,NaN,2015-02-24 11:15:36 -0800,NaN,Pacific Time (US & Canada),virginamerica really aggressive blast obnoxiou...
4,570300817074462722,0,1.0000,Can't Tell,1.0000,Virgin America,NaN,jnardino,NaN,0,@VirginAmerica and it's a really big bad thing...,NaN,2015-02-24 11:14:45 -0800,NaN,Pacific Time (US & Canada),virginamerica really big bad thing


# Text vectorization - Training/Test Set construction

In [6]:
X = df['clean_text']
y = df['source']

tfidf = TfidfVectorizer()
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=1, stratify = y)

t0 = time.time()
X_train_vec = tfidf.fit_transform(X_train)
#print("== TF-IDF VOCAB X_TRAIN =============================================================\n")
#print(tfidf.vocabulary_)

X_test_vec = tfidf.transform(X_test)
#print("== TF-IDF VOCAB X_TEST  =============================================================\n")
#print(tfidf.vocabulary_)

t1 = time.time()

print("Text vectorization duration:", t1 - t0)
print("Training set dimensionality: ", X_train_vec.shape)
print("Test set dimensionality: ", X_test_vec.shape)



Text vectorization duration: 0.15700006484985352
Training set dimensionality:  (10248, 9849)
Test set dimensionality:  (4392, 9849)


# Models on the original dataset

In [7]:
for clf in models:
    t0 = time.time()
    clf.fit(X_train_vec, y_train)
    t1 = time.time()
    print("=================================================================================================")
    print("\t === CLASSIFIER:", clf, "- TARGET SPACE: original")
    print("\t === Model training:", t1 - t0, "sec")

    y_predicted = clf.predict(X_test_vec)

    print("\t ========= Classification Report")
    print("\t", classification_report(y_test, y_predicted))
    print("=================================================================================================")


	 === CLASSIFIER: KNeighborsClassifier(n_neighbors=10) - TARGET SPACE: original
	 === Model training: 0.0009999275207519531 sec
	 ========= Classification Report
	               precision    recall  f1-score   support

           0       0.80      0.87      0.83      2753
           1       0.49      0.42      0.45       930
           2       0.66      0.56      0.60       709

    accuracy                           0.72      4392
   macro avg       0.65      0.61      0.63      4392
weighted avg       0.71      0.72      0.71      4392

	 === CLASSIFIER: LogisticRegression(max_iter=300) - TARGET SPACE: original
	 === Model training: 0.7300000190734863 sec
	 ========= Classification Report
	               precision    recall  f1-score   support

           0       0.81      0.94      0.87      2753
           1       0.67      0.50      0.57       930
           2       0.78      0.55      0.65       709

    accuracy                           0.78      4392
   macro avg       0.75   

# Models on the reduced dimensional spaces

In [8]:
reduced_spaces = [ 10, 100, 1000, 10000 ]
for target_space in reduced_spaces:
    print("========== Working with target space of", target_space, "dimensions ============")
    SVD = TruncatedSVD(n_components=target_space, random_state=42)
    X_train_red = SVD.fit_transform(X_train_vec)
    X_test_red = SVD.transform(X_test_vec)

    print("\tInput space dimensionality (training):", X_train_red.shape, X_train_vec.shape)
    print("\tInput space dimensionality (testing):", X_test_red.shape, X_test_vec.shape)

    for clf in models:
        t0 = time.time()
        clf.fit(X_train_red, y_train)
        t1 = time.time()
        print("=================================================================================================")
        print("\t === CLASSIFIER:", clf, "- TARGET SPACE:", target_space)
        print("\t === Model training:", t1 - t0, "sec")

        y_predicted = clf.predict(X_test_red)

        print("\t === Classification Report")
        print(classification_report(y_test, y_predicted))
        print("=================================================================================================")


========== Working with target space of 10 dimensions ============
	Input space dimensionality (training): (10248, 10) (10248, 9849)
	Input space dimensionality (testing): (4392, 10) (4392, 9849)
	 === CLASSIFIER: KNeighborsClassifier(n_neighbors=10) - TARGET SPACE: 10
	 === Model training: 0.0409998893737793 sec
	 === Classification Report
              precision    recall  f1-score   support

           0       0.75      0.92      0.83      2753
           1       0.52      0.32      0.39       930
           2       0.62      0.42      0.50       709

    accuracy                           0.71      4392
   macro avg       0.63      0.55      0.57      4392
weighted avg       0.68      0.71      0.68      4392

	 === CLASSIFIER: LogisticRegression(max_iter=300) - TARGET SPACE: 10
	 === Model training: 0.1770002841949463 sec
	 === Classification Report
              precision    recall  f1-score   support

           0       0.71      0.96      0.82      2753
           1       0.55 

C:\Users\Leo\anaconda3\lib\site-packages\sklearn\neural_network\_multilayer_perceptron.py:614: ConvergenceWarning: Stochastic Optimizer: Maximum iterations (200) reached and the optimization hasn't converged yet.
  warnings.warn(


	 === CLASSIFIER: MLPClassifier(hidden_layer_sizes=(128, 16), random_state=42) - TARGET SPACE: 100
	 === Model training: 12.201000213623047 sec
	 === Classification Report
              precision    recall  f1-score   support

           0       0.80      0.84      0.82      2753
           1       0.52      0.48      0.50       930
           2       0.61      0.56      0.58       709

    accuracy                           0.72      4392
   macro avg       0.64      0.63      0.63      4392
weighted avg       0.71      0.72      0.71      4392

========== Working with target space of 1000 dimensions ============
	Input space dimensionality (training): (10248, 1000) (10248, 9849)
	Input space dimensionality (testing): (4392, 1000) (4392, 9849)
	 === CLASSIFIER: KNeighborsClassifier(n_neighbors=10) - TARGET SPACE: 1000
	 === Model training: 0.012999773025512695 sec
	 === Classification Report
              precision    recall  f1-score   support

           0       0.75      0.57      

ValueError: n_components must be < n_features; got 10000 >= 9849